# Benchmark for signal representation 

- Multifractal analysis

- Discrete Fourier Transform (DFT)
- Spectrogram
- Local symbolic features
- SAX representation
- Approximate entropy

ML

- Autoencoder

- RNN
- LSTM



In [101]:
import numpy as np

import pandas as pd
import torch
import pywt

import orjson

In [102]:
import pymultifracs.mfa as mfa
from pymultifracs.utils import build_q_log

In [103]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# import pandas_datareader as pdr
import seaborn as sns
from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg, ar_select_order


In [104]:
# Gaussian process covariance function
def covariance_function(x):
    return 1. * np.exp(-((x / 10.)**2))

# Gaussian process simulation function
def sample_gaussian_process(c):
    """
    c array representing covariance function computed in x
    """
    white_noise = np.random.normal(0, 1, len(c))

    fc = np.fft.fft(np.fft.ifftshift(c))
    fw = np.fft.fft(white_noise)
    fz = np.sqrt(fc) * fw

    return np.fft.fftshift(np.fft.ifft(fz)).real

c = covariance_function(np.arange(0,100,1))

X = torch.FloatTensor( np.array([sample_gaussian_process(c) for _ in range(50)]))

In [105]:
class Transformator():
    def __init__(self) -> None:
        pass


In [106]:
%%script false --no-raise-error
class DataTransform():
    def __init__(self) -> None:
        self.transformed_X = None
        pass

    @property
    def X(self):
        return self.transformed_X
    @property
    def shape(self):
        return self.transformed_X.shape
    
    # @staticmethod
    def identity(self,X):
        return torch.FloatTensor(X)
    
    # @staticmethod
    def fourier(self,X, new_dimentsion = None):
        # Compute the Fourier transform
        fourier_transform = torch.fft.fft(X,n=new_dimentsion)
        
        # Compute the modulus (magnitude) of the Fourier transform
        modulus = torch.abs(fourier_transform)
        
        return modulus
    
    # @staticmethod
    def wavedec(self,X, wavelet='db1', mode='symmetric'):
        
        # Convert PyTorch tensor to NumPy array
        array = X.numpy()
    
        # Compute the wavelet transform
        coeffs = pywt.wavedec(array, wavelet, mode=mode)

        # coeffs_array = pywt.coeffs_to_array(coeffs)
        # # Convert the result back to PyTorch tensors
        coeffs_torch = [torch.tensor(c) for c in coeffs]
       
    
        return coeffs_torch
    
    def wavedec(self,X, wavelet='db1', mode='symmetric'):
        
        # Convert PyTorch tensor to NumPy array
        array = X.numpy()
    
        # Compute the wavelet transform
        coeffs = pywt.wavedec(array, wavelet, mode=mode)

        coeffs_array = pywt.coeffs_to_array(coeffs)
        # # Convert the result back to PyTorch tensors
        # coeffs_torch = [torch.tensor(c) for c in coeffs]
        # # coeffs_torch = torch.FloatTensor(coeffs_array)
        # return torch.cat(coeffs_torch,dim=-1)
        coeffs_array = pywt.coeffs_to_array(coeffs)
    
        return torch.FloatTensor(coeffs_array[0])
    
    def wavedec(self,X,level=4, wavelet='db1', mode='symmetric'):
        
        # Convert PyTorch tensor to NumPy array
        array = X.numpy()
    
        # Compute the wavelet transform
        coeffs = pywt.wavedec(array, wavelet, mode=mode,level=level)

        # coeffs_array = pywt.coeffs_to_array(coeffs)
        # # Convert the result back to PyTorch tensors
        coeffs_torch = [torch.tensor(c) for c in coeffs[:1]]
       
        return torch.cat(coeffs_torch,dim=-1)
    # @staticmethod
    def dwt(self,X, wavelet='db1', mode='symmetric'):
        # Convert PyTorch tensor to NumPy array
        array = X.numpy()
    
        # Compute the wavelet transform
        coeffs = pywt.dwt(array, wavelet, mode=mode)
    
        # Convert the result back to PyTorch tensors
        coeffs_torch = [torch.tensor(c) for c in coeffs]
    
        return torch.cat(coeffs_torch,dim=-1)
    
    # @staticmethod
    def get_ar_coefficients(self, X, k):
        n, p = X.shape
        ar_coefficients = np.zeros((n, k))

        for i in range(n):
            # Fit the AR model to the i-th time series
            model = AutoReg(X[i], lags=k).fit()
            
            # Extract the coefficients (excluding the intercept term if present)
            ar_coefficients[i] = model.params[1:k+1]  # assuming the intercept is model.params[0]

        return ar_coefficients
    
    # @staticmethod
    def autoreg(self,X,k):
        return torch.tensor(self.get_ar_coefficients(X,k))
    
    def shannon_encoding(self,X,level,wavelet='db1',mode='symmetric'):

        def compute_shannon_entropy(signal):
            """Here, we will compute nonnormalized shannon entropy.
            We will basically implement MATLAB function 'wentropy'.
            Input: signal should be 1D numpy array."""
            return -np.nansum(signal**2 * np.log(signal**2)) # nansum to exclude pesky terms like "0*np.log(0)"
        

        n_examples = X.shape[0]
        wp = pywt.WaveletPacket(X[0,:], wavelet = "sym8", maxlevel = 3)
        packet_names = [node.path for node in wp.get_level(3, "natural")]  # Packet node names. 
        
        feature_matrix_wav_packet_entropy = np.repeat(np.nan, n_examples*8).reshape(n_examples,8)
        for i in range(len(X)):
            wp = pywt.WaveletPacket(X[i,:], wavelet = "sym8", maxlevel = 3) # Wavelet packet transformation
            for j in range(len(packet_names)):
                new_wp = pywt.WaveletPacket(data = None, wavelet = "sym8", maxlevel = 3)
                new_wp[packet_names[j]] = wp[packet_names[j]].data
                reconstructed_signal = new_wp.reconstruct(update = False) # Signal reconstruction from wavelet packet coefficients
                feature_matrix_wav_packet_entropy[i,j] = compute_shannon_entropy(reconstructed_signal) # Entropy of reconstructed signal for every node
                # print(reconstructed_signal)
                # print(feature_matrix_wav_packet_entropy[i,j])
        return feature_matrix_wav_packet_entropy
    
    def wavelet_leaders(self,X,j1=2,j2=6):
        n = X.shape[0] if X.ndim > 1 else 1
        transformed_X = -  np.ones((n,2))
        for i in range(X.shape[0]):
            dwt, lwt = mfa.mf_analysis_full(X[i],
                scaling_ranges=[(2, 6)],
                q=build_q_log(1, 10, 20),
                n_cumul=2,
                p_exp=np.inf,
                gamint=0.0
            )
            sf, cumul, mfs, hmin = lwt
            transformed_X[i,:] = sf.H.item(), cumul.log_cumulants[1].item()

        return transformed_X

Couldn't find program: 'false'


In [108]:
class DataTransform:
    def __init__(self) -> None:
        self.transformed_X = None

    @property
    def X(self):
        return self.transformed_X
    
    @property
    def shape(self):
        if self.transformed_X is not None:
            return self.transformed_X.shape
        else:
            return None

    @staticmethod
    def identity(X):
        return torch.FloatTensor(X)
    
    @staticmethod
    def fourier(X, new_dimension=None):
        fourier_transform = torch.fft.fft(X, n=new_dimension)
        modulus = torch.abs(fourier_transform)
        return modulus
    
    @staticmethod
    def wavedec(X, level=4, wavelet='db1', mode='symmetric'):
        array = X.numpy()
        coeffs = pywt.wavedec(array, wavelet, mode=mode, level=level)
        coeffs_torch = [torch.tensor(c) for c in coeffs[:1]]
        return torch.cat(coeffs_torch, dim=-1)
    
    @staticmethod
    def dwt(X, wavelet='db1', mode='symmetric'):
        array = X.numpy()
        coeffs = pywt.dwt(array, wavelet, mode=mode)
        coeffs_torch = [torch.tensor(c) for c in coeffs]
        return torch.cat(coeffs_torch, dim=-1)
    
    @staticmethod
    def get_ar_coefficients(X, k):
        n, p = X.shape
        ar_coefficients = np.zeros((n, k))
        for i in range(n):
            model = AutoReg(X[i], lags=k).fit()
            ar_coefficients[i] = model.params[1:k+1] 
        return ar_coefficients
    
    @staticmethod
    def autoreg(X, k):
        return torch.tensor(DataTransform.get_ar_coefficients(X, k))
    
    @staticmethod
    def shannon_encoding(X, level=4, wavelet='db1', mode='symmetric'):
        def compute_shannon_entropy(signal):
            return -np.nansum(signal**2 * np.log(signal**2))
        
        n_examples = X.shape[0]
        wp = pywt.WaveletPacket(X[0, :], wavelet="sym8", maxlevel=3)
        packet_names = [node.path for node in wp.get_level(3, "natural")]
        
        feature_matrix_wav_packet_entropy = np.full((n_examples, 8), np.nan)
        for i in range(len(X)):
            wp = pywt.WaveletPacket(X[i, :], wavelet="sym8", maxlevel=3)
            for j in range(len(packet_names)):
                new_wp = pywt.WaveletPacket(data=None, wavelet="sym8", maxlevel=3)
                new_wp[packet_names[j]] = wp[packet_names[j]].data
                reconstructed_signal = new_wp.reconstruct(update=False)
                feature_matrix_wav_packet_entropy[i, j] = compute_shannon_entropy(reconstructed_signal)
        return feature_matrix_wav_packet_entropy
    
    @staticmethod
    def wavelet_leaders(X, j1=2, j2=6):
        n = X.shape[0] if X.ndim > 1 else 1
        transformed_X = -np.ones((n, 2))
        for i in range(X.shape[0]):
            dwt, lwt = mfa.mf_analysis_full(
                X[i],
                scaling_ranges=[(j1, j2)],
                q=mfa.build_q_log(1, 10, 20),
                n_cumul=2,
                p_exp=np.inf,
                gamint=0.0
            )
            sf, cumul, mfs, hmin = lwt
            transformed_X[i, :] = sf.H.item(), cumul.log_cumulants[1].item()
        return transformed_X

    def apply_transformation(self, X, transformation_name, **kwargs):
        transformation_methods = {
            'identity': self.identity,
            'fourier': self.fourier,
            'wavedec': self.wavedec,
            'dwt': self.dwt,
            'autoreg': self.autoreg,
            'shannon_encoding': self.shannon_encoding,
            'wavelet_leaders': self.wavelet_leaders,
        }
        
        if transformation_name in transformation_methods:
            return transformation_methods[transformation_name](X, **kwargs)
        else:
            raise ValueError(f"Transformation {transformation_name} not recognized.")


In [109]:
X.shape

torch.Size([50, 100])

In [110]:
a,b,c,d,e,f,g = pywt.wavedec(X.numpy(),wavelet='db1')


In [111]:
len(pywt.dwt(X.numpy(),wavelet='db1'))

2

In [112]:
def wavelet_transform(tensor, wavelet='db1', mode='symmetric'):
    # Convert PyTorch tensor to NumPy array
    array = tensor.numpy()
    
    # Compute the wavelet transform
    coeffs = pywt.wavedec(array, wavelet, mode=mode)
    
    # Convert the result back to PyTorch tensors
    coeffs_torch = [torch.tensor(c) for c in coeffs]
    # coeffs_array = pywt.coeffs_to_array(coeffs)
    # # Convert the result back to PyTorch tensors
    # coeffs_torch = [torch.tensor(c) for c in coeffs]
    # # coeffs_torch = torch.FloatTensor(coeffs_array)
    
    return coeffs_torch

def wavelet_transform2(tensor, wavelet='db1', mode='symmetric'):
    # Convert PyTorch tensor to NumPy array
    array = tensor.numpy()
    
    # Compute the wavelet transform
    coeffs = pywt.wavedec(array, wavelet, mode=mode)
    
    coeffs_array = pywt.coeffs_to_array(coeffs)
    
    return torch.FloatTensor(coeffs_array[0])
# Example usage:
tensor = torch.tensor(X[0])
wavelet_coeffs = wavelet_transform(tensor)
print(wavelet_coeffs)
for i, coeff in enumerate(wavelet_coeffs):
    print(f'Coefficient {i}: {coeff}')

tensor = torch.tensor([1.0, 2.0, 3.0, 4.0])
wavelet_coeffs = wavelet_transform2(tensor)
print(wavelet_coeffs)
for i, coeff in enumerate(wavelet_coeffs):
    print(f'Coefficient {i}: {coeff}')


[tensor([1.8800, 0.9046]), tensor([-0.2584, -1.5160]), tensor([ 0.2779,  5.3936, -3.4864,  0.0000]), tensor([-0.9118,  0.2720,  1.3084, -3.2664,  2.6224, -0.6204,  0.0000]), tensor([-0.6910, -0.0629,  0.5940,  1.0400, -0.9947,  4.4571, -0.0081, -1.3618,
         1.3416, -2.0219, -1.1804,  1.4271,  0.0000]), tensor([-0.2979, -0.0144,  1.2894, -0.7946,  2.3672, -0.6655,  1.4477,  0.0663,
        -0.8370, -0.4419,  1.8736,  0.3160,  0.1559, -0.7624,  0.1843, -0.8321,
         2.4465,  2.3645, -0.6316, -0.3934, -1.5510, -1.4870,  0.6557,  0.3796,
        -0.7889]), tensor([-0.3484,  0.3363,  0.2300,  0.0551, -0.2527,  0.3295, -0.1840, -1.2970,
        -0.1129,  0.6218, -0.8449,  0.0310,  0.0173,  1.0361,  0.4858, -0.5414,
        -0.4050,  0.0672,  0.5786, -1.8602,  0.6107,  0.9640,  0.3565, -0.2018,
         0.5241, -0.6477,  0.8863,  0.6659, -0.0942,  0.4226, -0.3055,  0.3139,
        -0.8859,  0.3828,  0.6621,  0.9664, -0.5670, -0.1343,  0.6387, -0.1333,
         0.4540,  0.3731, -0.534

C:\Users\aejog\AppData\Local\Temp\ipykernel_11836\206756173.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(X[0])


In [113]:
pywt.wavelist()

['bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'cgau1',
 'cgau2',
 'cgau3',
 'cgau4',
 'cgau5',
 'cgau6',
 'cgau7',
 'cgau8',
 'cmor',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'dmey',
 'fbsp',
 'gaus1',
 'gaus2',
 'gaus3',
 'gaus4',
 'gaus5',
 'gaus6',
 'gaus7',
 'gaus8',
 'haar',
 'mexh',
 'morl',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',

In [114]:
class Identity(DataTransform):
    def __init__(self, X) -> None:
        super().__init__()
        self.transformed_X = torch.FloatTensor(X)

## Load data

In [115]:
# # Read JSON data using orjson
# with open('mydata-mitbih.json', 'r') as f:
#     data = orjson.loads(f.read())

# # Convert the data to a pandas DataFrame
# df = pd.DataFrame(data)

In [116]:
X_df= pd.read_json('mydata-mitbih-small.json')

In [117]:
X_data = np.array(X_df.sample(frac=1))
Y_data = X_data[:,-1]
X_data = X_data[:,:-1]

Y_data = 1.*(Y_data > 0.)



In [118]:
X = X_data
y = Y_data

In [119]:
import h5py

In [120]:
def load_dict_from_hdf5(filename):
    """
    Load a dictionary of lists of arrays from an HDF5 file.

    Parameters:
    filename (str): The name of the HDF5 file to load the data from.

    Returns:
    dict: The dictionary containing lists of arrays.
    """
    data_dict = {}
    with h5py.File(filename, 'r') as hdf5_file:
        for key in hdf5_file.keys():
            group = hdf5_file[key]
            data_dict[key] = np.array([group[f'array_{i}'][...] for i in range(len(group))])
    return data_dict

data_dict = load_dict_from_hdf5('final_data.h5')

In [121]:
data_dict.keys()

dict_keys(['bidmc-cong', 'mit-bih-ar', 'mit-bih-no'])

In [122]:
X = []
y = []

target = 0
skip = True
for key,value in data_dict.items():
    # if  skip:
    #     skip  = False
    #     continue
    n = value.shape[0]
    X.append(torch.tensor(np.array(value)))
    y.append(torch.ones(n,dtype=int)*target)
    target+=1

X = torch.cat(X).squeeze()
y = torch.cat(y)

In [123]:
# X = DataTransform().(X)

In [124]:

array = np.array(X)

# Compute the wavelet transform
coeffs = pywt.wavedec(array, 'db2', mode='symmetric')
# print(coeffs)
# coeffs_array = pywt.coeffs_to_array(coeffs)
# # Convert the result back to PyTorch tensors
coeffs_torch = [torch.tensor(c) for c in coeffs]
#coeffs_torch = torch.FloatTensor(coeffs_array)
torch.cat(coeffs_torch,dim=-1).shape

torch.Size([840, 65032])

In [125]:
DataTransform().fourier(X,250).shape

torch.Size([840, 250])

In [126]:
from modwt import modwt, modwtmra
import pandas as pd

# dont #work
# wt = modwt(np.array(X), 'db2', 5)
# wtmra = modwtmra(wt, 'db2')

In [127]:
# from collections import namedtuple

In [128]:
# type(dwt)

In [129]:
# import pymultifracs.mfa as mfa
# from pymultifracs.utils import build_q_log

In [130]:
dwt, lwt = mfa.mf_analysis_full(X[10],
    scaling_ranges = [(2,6)],
    q = build_q_log(1, 10, 20),
    n_cumul=2,
    p_exp=np.inf,
    gamint=0.0
)
lwt_sf, lwt_cumul, lwt_mfs, hmin = lwt

In [131]:
# # X = np.array(X)
# n = X.shape[0] if X.ndim > 1 else 1
# transformed_X = -  np.ones((n,2))
# for i in range(X.shape[0]):
#     dwt, lwt = mfa.mf_analysis_full(X[i],
#     scaling_ranges = [(2,6)],
#     q = build_q_log(1, 10, 20),
#     n_cumul=2,
#     p_exp=np.inf,
#     gamint=0.0
# )
#     sf, cumul, mfs, hmin = lwt
#     transformed_X[i,:] = sf.H.item(), cumul.log_cumulants[1].item()

# transformed_X


In [132]:
X[3].shape

torch.Size([65000])

In [133]:
# len(dwt[])
# DataTransform().wavelet_leaders(X[0:100],2,6)

In [134]:
# np.fft.fft()
# DataTransform().wavelet_leaders(X[0:100],2,3)


In [135]:
# X = DataTransform().fourier(X,250)
# X = DataTransform().wavedec(X,wavelet='db2')
X = DataTransform().shannon_encoding(np.array(X),level=5)


In [136]:
X.shape

(840, 8)

In [137]:
%%script false --no-raise-error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg

# Generate a sample time series data
np.random.seed(42)
n = 100
time_series = np.sin(np.linspace(0, 20, n)) + np.random.normal(scale=0.5, size=n)

# Plot the time series
plt.plot(time_series)
plt.title('Sample Time Series Data')
plt.show()

# Fit an autoregressive model to the time series data
# Using lag value of 4 for demonstration
lag = 4
model = AutoReg(time_series, lags=lag).fit()

# Print the coefficients
print("Coefficients of the AR model:")
print(model.params)

# Predicting future values
pred = model.predict(start=len(time_series), end=len(time_series)+10)
print("\nPredicted values:")
print(pred)


Couldn't find program: 'false'


In [138]:
# housing = np.array(X)
# mod = AutoReg(housing[0], 3)
# res = mod.fit()
# print(res.summary())

## Data prep

In [139]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader

# Load and preprocess the dataset
# iris = datasets.load_iris()
# X, y = iris.data, iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


C:\Users\aejog\AppData\Local\Temp\ipykernel_11836\4202145339.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.long)
C:\Users\aejog\AppData\Local\Temp\ipykernel_11836\4202145339.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [140]:
iris = datasets.load_iris()
print(iris.target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [141]:
# Define the classifiers
class FCNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class SimpleCNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=2, stride=1)
        self.fc1 = nn.Linear(16 * (input_dim - 1), output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

class SimpleRNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_dim, 50, batch_first=True)
        self.fc1 = nn.Linear(50, output_dim)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc1(x[:, -1, :])
        return x

# Function to train and evaluate the classifiers
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    model.eval()
    y_pred = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.numpy())
    
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Initialize models, criteria, and optimizers
input_dim = X.shape[1]
output_dim = len(set(y))

models = {
    'FCNN': FCNN(input_dim, output_dim),
    'CNN': SimpleCNN(input_dim, output_dim),
    # 'RNN': SimpleRNN(input_dim, output_dim)
}

criteria = {
    'FCNN': nn.CrossEntropyLoss(),
    'CNN': nn.CrossEntropyLoss(),
    # 'RNN': nn.CrossEntropyLoss()
}

optimizers = {
    'FCNN': optim.Adam(models['FCNN'].parameters(), lr=0.01),
    'CNN': optim.Adam(models['CNN'].parameters(), lr=0.01),
    # 'RNN': optim.Adam(models['RNN'].parameters(), lr=0.01)
}

# Train and evaluate each model
for name, model in models.items():
    criterion = criteria[name]
    optimizer = optimizers[name]
    accuracy = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer)
    print(f"{name} Accuracy: {accuracy:.4f}")


FCNN Accuracy: 1.0000
CNN Accuracy: 1.0000


## Tests

In [142]:
# wt

In [143]:
# wtmra

In [144]:
# X = 
# y = 

In [145]:
# Load dataset
# iris = datasets.load_iris()
# X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X[281:], y[281:] - 1, test_size=0.3, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [146]:

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


# Initialize classifiers
classifiers = {
    
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cross_val = cross_val_score(clf, X, y, cv=5)
    results[name] = {"Accuracy": accuracy, "Cross-Validation Score": cross_val.mean()}

# Print results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  Cross-Validation Score: {metrics['Cross-Validation Score']:.4f}")
    print()


Decision Tree:
  Accuracy: 1.0000
  Cross-Validation Score: 0.9976

Random Forest:
  Accuracy: 1.0000
  Cross-Validation Score: 1.0000

SVM:
  Accuracy: 1.0000
  Cross-Validation Score: 1.0000



In [147]:
stop

NameError: name 'stop' is not defined

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score

clf = SVC()
results = {}
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cross_val = cross_val_score(clf, X, y, cv=5)
results[name] = {"Accuracy": accuracy, "Cross-Validation Score": cross_val.mean()}

# Print results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  Cross-Validation Score: {metrics['Cross-Validation Score']:.4f}")
    print()

CNN:
  Accuracy: 0.8452
  Cross-Validation Score: 0.8262



In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


# Initialize classifiers
classifiers = {
    # "Logistic Regression": LogisticRegression(max_iter=1),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "k-NN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cross_val = cross_val_score(clf, X, y, cv=5)
    results[name] = {"Accuracy": accuracy, "Cross-Validation Score": cross_val.mean()}

# Print results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  Cross-Validation Score: {metrics['Cross-Validation Score']:.4f}")
    print()


KeyboardInterrupt: 